In [8]:
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation

# File paths
puzzle_info_path = 'puzzle_info.csv'
puzzles_path = 'puzzles.csv'
sample_submission_path = 'sample_submission.csv' # change name across board
my_submission_path = 'submission.csv'

# Loading the data
puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)
my_submission_df = pd.read_csv(my_submission_path)
puzzles_df.head()

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0


# Globe Puzzles #
A `globe` puzzle is a sphere with cuts along lines of latitude and longitude. If you poke a hole at the North and South poles, cut along the meridian, and spread it out, it will look like a grid:

```
  | f0  f1  f2  f3  f4  f5  f6  f7
--+-------------------------------
r0| 0   1   2   3   4   5   6   7
r1| 8   9   10  11  12  13  14  15
r2| 16  17  18  19  20  21  22  23
r3| 24  25  26  27  28  29  30  31
```

More precisely, a `globe_M/N` is a sphere with `M` lateral cuts (through latitude) and N radial cuts (through longitude). This gives a `(M + 1) x (2 N)` grid of positions. You could format a color state into a grid like this with something like `np.asarray(state).reshape(m+1, 2*n)`. The above is a `globe_3/4` puzzle.

In [9]:
# Get a solution_state
ss_globe34 = puzzles_df[puzzles_df["puzzle_type"] == 'globe_1/8'].iloc[0]['initial_state'].split(';')

# Reshape into a grid
np.asarray(ss_globe34).reshape(1+1, 2*8)

array([['I', 'P', 'O', 'A', 'A', 'D', 'F', 'L', 'J', 'M', 'G', 'M', 'P',
        'F', 'E', 'J'],
       ['E', 'B', 'O', 'G', 'H', 'D', 'N', 'L', 'N', 'I', 'B', 'K', 'C',
        'C', 'K', 'H']], dtype='<U1')

In [10]:
puzzles_df[puzzles_df["puzzle_type"] == 'globe_3/4']

,id,puzzle_type,solution_state,initial_state,num_wildcards
358,358,globe_3/4,A;A;C;C;E;E;G;G;A;A;C;C;E;E;G;G;B;B;D;D;F;F;H;...,C;F;B;F;H;H;E;D;H;E;G;A;G;C;E;D;C;B;B;A;F;D;F;...,2
359,359,globe_3/4,A;A;C;C;E;E;G;G;A;A;C;C;E;E;G;G;B;B;D;D;F;F;H;...,D;B;G;B;F;E;D;C;F;B;F;E;B;G;H;A;H;A;D;C;E;C;D;...,4
360,360,globe_3/4,A;A;C;C;E;E;G;G;A;A;C;C;E;E;G;G;B;B;D;D;F;F;H;...,E;E;F;A;D;G;F;H;C;F;A;C;G;B;H;H;G;D;B;A;F;D;E;...,2
361,361,globe_3/4,A;A;C;C;E;E;G;G;A;A;C;C;E;E;G;G;B;B;D;D;F;F;H;...,C;E;G;H;E;F;F;H;G;E;F;E;B;H;A;C;H;B;C;A;G;F;D;...,0
362,362,globe_3/4,A;A;C;C;E;E;G;G;A;A;C;C;E;E;G;G;B;B;D;D;F;F;H;...,F;H;D;C;D;E;E;A;A;E;E;F;C;H;D;D;A;H;F;B;B;G;G;...,0
363,363,globe_3/4,A;A;C;C;E;E;G;G;A;A;C;C;E;E;G;G;B;B;D;D;F;F;H;...,F;D;B;G;H;C;A;C;C;A;H;E;C;F;E;A;G;D;F;B;H;D;B;...,0
364,364,globe_3/4,A;A;C;C;E;E;G;G;A;A;C;C;E;E;G;G;B;B;D;D;F;F;H;...,E;G;G;H;C;B;B;F;G;H;E;A;C;F;B;C;G;F;E;D;B;H;A;...,0
365,365,globe_3/4,A;A;C;C;E;E;G;G;A;A;C;C;E;E;G;G;B;B;D;D;F;F;H;...,A;G;B;E;F;G;H;A;B;E;F;C;G;D;A;D;H;C;G;A;H;F;E;...,0
366,366,globe_3/4,A;A;C;C;E;E;G;G;A;A;C;C;E;E;G;G;B;B;D;D;F;F;H;...,B;C;F;G;B;D;E;H;E;C;F;H;G;B;H;A;G;C;B;D;A;D;E;...,0
367,367,globe_3/4,A;A;C;C;E;E;G;G;A;A;C;C;E;E;G;G;B;B;D;D;F;F;H;...,D;B;F;G;F;C;A;D;C;E;A;B;F;G;F;C;E;H;A;H;G;D;D;...,0


In [11]:
# Hemisphere operation (180 flip), latitude rotation

# The top/bottom, top-1/bottom+1.... independent
# all the elements in start = end for palindromic rows
# the number of "displaced elements" on top = on bottom
# in 3 moves, we can do a cyclic rotation of half of the perm for top/bot
# all we need to do is ensure the bottom row (N24..N31 end up exactly where they are in the solution state) -> all elements of the top rwo are forced (except the case where answer is just a rotation of top / bottom, all hemisphere operations cancel)
# the order of elements on the top will always be in reverse, the order on the bottom is always opposite

# For globe 1/8, we can reduce to only three moves -> up shift, down shift, hemisphere (at index 0) shift

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile
import pandas as pd

In [ ]:
@jit(nopython=True, parallel = True, fastmath = True)
def hash_perm(perm):
    base = 9973
    modb = 1000000007
    modc = 1000000009

    B, C = 0, 0
    for i in perm:
        B = (B * base) % modb + i
        C = (C * base) % modc + i

    return (B, C)

In [ ]:
# 1e6 ~ 2 seconds
mx_mem = int(1e7)
mem_idx = 0

last_state = np.zeros(mx_mem, dtype=int)
last_move = np.zeros(mx_mem, dtype=int)

cur_state = [np.zeros(32, dtype=int) for i in range(mx_mem)]

print(last_state)

In [ ]:
# M = np.array([all_moves['r0'], all_moves['r1'], all_moves['f0']])
M = np.array([[5, 4, 2, 3, 1, 0, 6, 7], [3, 0, 1, 2, 4,5,6,7], [0,1,2,3, 7,4,5,6]])
M_names = ['f0', 'r0', 'r1']

In [ ]:
# M = np.array([all_moves['r0'], all_moves['r1'], all_moves['f0']])
# 6 : 6
M = np.array([[8, 7, 6, 3,4,5, 2,1,0,9,10,11], [5,0,1,2,3,4,6,7,8,9,10,11], [0,1,2,3,4,5,11,6,7,8,9,10]])
M_names = ['f0', 'r0', 'r1']

In [20]:
%%time

from collections import deque

# @jit(nopython=True, fastmath = True)
def BFS(last_state, last_move, cur_state, M):
    mem_idx = 1
    initial_state = np.arange(12)
    visited = dict()
    visited[hash_perm(initial_state)] = 0

    Q = deque([initial_state])

    i = 0
    while len(Q) > 0:
        state = Q.popleft()
        
        for j in range(3):
            m = M[j]
            next_state = state[m]
            # next_state = m(state)
            next_hash = hash_perm(next_state)
            
            if(next_hash in visited):
                continue
                
            visited[next_hash] = mem_idx
            
            # last_state[mem_idx] = i
            # last_move[mem_idx] = j
            
            mem_idx += 1

            Q.append(next_state)
        
        i += 1
        
    return visited

CPU times: user 14 µs, sys: 1 µs, total: 15 µs
Wall time: 19.1 µs


In [24]:
def dig_through_memory(idx):
    res = []
    while idx != 0:
        res.append(last_move[idx])
        idx = last_state[idx]

    res = list(reversed(res))

    res_names = []
    for i in res:
        res_names.append(M_names[i])
    return res_names

dig_through_memory(a[test_hash])

NameError: name 'a' is not defined

In [33]:
idx = 349	
N = 1+1
M = 2*8

In [36]:
all_moves = {}
print("globe_1/8")
for m, p in puzzle_info_df.loc['globe_1/8', 'allowed_moves'].items():
    all_moves[m] = Permutation(p)
    all_moves['-' + m] = Permutation(p) ** -1

# gstate = all_moves['-r0'](gstate)
gstate = np.array([i for i in range(32)])
print(np.asarray(gstate).reshape(1+1, 2*8))

globe_1/8


KeyError: 'globe_1/8'

In [30]:
def expand(path, n = 2, m = 16):    
    rows = [[] for _ in range(n)]
    
    def expand_f(x):
        for i in range(n):
            rows[i].append(x)
            rows[i].append('f0')
            rows[i].append(m - x)
    
    for i in path:
        if 'f' in i:
            if '-' in i:
                expand_f(int(i[2:]))
            else:
                expand_f(int(i[1:]))
        else:
            for j in range(n):
                temp = 'r' + str(j)

                if temp == i:
                    rows[j].append(1)
                    
                temp = '-r' + str(j)

                if temp == i:
                    rows[j].append(m-1)

    Rrows = [[] for _ in range(n)]

    for r in range(n):
        run = 0
        for i in rows[r]:
            if type(i) == type(0):
                run += i
            else:
                Rrows[r].append(run)
                Rrows[r].append('f0')
                run = 0
            
        Rrows[r].append(run)
    
    return Rrows

def shrink(rows, n = 2, m = 16):
    for i in range(len(rows[0])):
        mn = int(1e9)
        for j in range(n):
            if type(rows[j][i]) == type(1):
                rows[j][i] %= m
            else:
                mn = min(mn, rows[j][i-1])
        
        for j in range(n):  
            if type(rows[j][i]) == type('string'):
                rows[j][i-1] -= mn
                rows[j][i] = 'f' + str(mn)
                rows[j][i+1] += mn
    
    final_moves = []
    for i in range(len(rows[0])):
        if type(rows[0][i]) == type('string'):
            final_moves.append(rows[0][i])
            continue

        for j in range(n):
            if rows[j][i] > m/2:
                final_moves = final_moves + ['-r' + str(j)] * (m - rows[j][i])
            else:
                final_moves = final_moves + ['r' + str(j)] * (rows[j][i])

    final_final_moves = []
    i = 0
    while i < len(final_moves):
        if i != len(final_moves) - 1 and ('f' in final_moves[i]) and final_moves[i] == final_moves[i+1]:
            i += 2
        else:
            final_final_moves.append(final_moves[i])
            i += 1
    
    return final_final_moves
        

# ex = expand(sample_submission['moves'][idx].split('.'), N, M)
ex = expand([
 '-r0',
'-r0',
'-r0',
'-r0',
'-r0',
'-r0',
'-r0',
'-r0',

    
 'f0',
 'r0',
 'f0',
    
 'r0',
 'r0',
'r0',
 'r0',
'r0',
 'r0',
  'r0',
 'r0',

'r0',
'r0',
 'r0',
'r0',
 'r0',
  'r0',
 'r0',

    'r1',
'r1',
 'r1',
'r1',
 'r1',
  'r1',
 'r1',

'-r0',
'-r0',
'-r0',
'-r0',
'-r0',
'-r0',
'-r0',
'-r0',

    
 'f0',
 'r0',
 'f0',
    
 'r0',
 'r0',
'r0',
 'r0',
'r0',
 'r0',
  'r0',
 'r0',
    
 ], 2, 16)
print(ex)
res = shrink(ex, 2, 16)
res

[[120, 'f0', 17, 'f0', 151, 'f0', 17, 'f0', 24], [0, 'f0', 16, 'f0', 23, 'f0', 16, 'f0', 16]]


['r0',
 'r0',
 'r0',
 'r0',
 'r0',
 'r0',
 'r0',
 'r0',
 'f0',
 'r0',
 'f0',
 'f7',
 'r0',
 'f7',
 '-r0',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1']

In [32]:
test_globe = np.array([i for i in range(2*8)])
cur_type = puzzles_df['puzzle_type'][idx]

for i in res:
    cur = cur[move_to_perm[cur_type][i]]

    print(cur)


NameError: name 'cur' is not defined

In [243]:
print(len(sample_submission['moves'][idx].split('.')))
print(len(res))

496
504


In [244]:
cur = np.array(puzzles['initial_state'][idx])
print(cur)
cur_type = puzzles['puzzle_type'][idx]

ALL = {}
temp = 0

calc = 0

for i in res:
    cur = cur[move_to_perm[cur_type][i]]

    if tuple(cur) in ALL:
        print(ALL[tuple(cur)], temp)
        calc += temp - ALL[tuple(cur)]

    if tuple(cur) not in ALL:
        ALL[tuple(cur)] = temp

    temp += 1

print('expected', np.array(puzzles['solution_state'][idx]))
print('got', cur)

['E' 'C' 'A' 'H' 'C' 'J' 'K' 'H' 'I' 'K' 'A' 'A' 'H' 'I' 'L' 'M' 'M' 'P'
 'I' 'N' 'C' 'L' 'M' 'K' 'G' 'O' 'F' 'D' 'J' 'N' 'F' 'B' 'J' 'G' 'P' 'L'
 'P' 'G' 'B' 'O' 'O' 'E' 'A' 'N' 'J' 'G' 'C' 'O' 'K' 'F' 'E' 'P' 'H' 'I'
 'N' 'M' 'E' 'F' 'D' 'L' 'B' 'D' 'D' 'B']
expected ['A' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'E' 'E' 'E' 'E' 'G' 'G' 'G' 'G' 'I' 'I'
 'I' 'I' 'K' 'K' 'K' 'K' 'M' 'M' 'M' 'M' 'O' 'O' 'O' 'O' 'B' 'B' 'B' 'B'
 'D' 'D' 'D' 'D' 'F' 'F' 'F' 'F' 'H' 'H' 'H' 'H' 'J' 'J' 'J' 'J' 'L' 'L'
 'L' 'L' 'N' 'N' 'N' 'N' 'P' 'P' 'P' 'P']
got ['A' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'E' 'E' 'E' 'E' 'G' 'G' 'G' 'G' 'I' 'I'
 'I' 'I' 'K' 'K' 'K' 'K' 'M' 'M' 'M' 'M' 'O' 'O' 'O' 'O' 'B' 'B' 'B' 'B'
 'D' 'D' 'D' 'D' 'F' 'F' 'F' 'F' 'H' 'H' 'H' 'H' 'J' 'J' 'J' 'J' 'L' 'L'
 'L' 'L' 'N' 'N' 'N' 'N' 'P' 'P' 'P' 'P']


In [231]:
print(len(res) - calc)

237


In [232]:
print(res[788:792])

[]
